# Nueral architecture search on minst dataset

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import autokeras as ak


2023-03-21 21:44:23.180790: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 21:44:24.013225: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-21 21:44:24.013286: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-21 21:44:25.820364: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
import torch
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)


Device: cuda


# Loading data
Here load the mnist data into training and testing samples

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)

(60000, 28, 28)
(60000,)
[5 0 4]


# Training
 Here train the autokeras built in image clasfier with mnist data after the training the validation accuracy of the architecture is 98 %

In [10]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=10)


# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Trial 1 Complete [00h 06m 51s]
val_loss: 0.042387738823890686

Best val_loss So Far: 0.042387738823890686
Total elapsed time: 00h 06m 51s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
1875/1875 [==============================] - 47s 25ms/step - loss: 0.1546 - accuracy: 0.9532
Epoch 2/10
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0721 - accuracy: 0.9779
Epoch 3/10
1875/1875 [==============================] - 48s 26ms/step - loss: 0.0573 - accuracy: 0.9822
Epoch 4/10
1875/1875 [==============================] - 52s 28ms/step - loss: 0.0505 - accuracy: 0.9842
Epoch 5/10
1875/1875 [==============================] - 48s 26ms/step - loss: 0.0436 - accuracy: 0.9863
Epoch 6/10
1875/1875 [==============================] - 50s 26ms/step - loss: 0.0403 - accuracy: 0.9871
Epoch 7/10
1875/1875 [==============================] - 49s 26ms/step - loss: 0.0353 - accuracy: 0.9894
Epoch 8/10
1875/1875 [==============================] - 48s 26ms/step - loss: 0.0349 - accuracy: 0.9888
Epoch 9/10
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0317 - accuracy: 0.9891
Epoch 10/10
1875/1875 [==============================] - 50s 27m

INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


313/313 [==============================] - 2s 5ms/step
[['7']
 ['2']
 ['1']
 ...
 ['4']
 ['5']
 ['6']]
313/313 [==============================] - 2s 5ms/step - loss: 0.0362 - accuracy: 0.9887
[0.03616759181022644, 0.9886999726295471]


# Customized Search Space

here only resnet archirtures will be searched for classification on mnist data will be train.

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [04h 43m 01s]
val_loss: 0.06486619263887405

Best val_loss So Far: 0.06486619263887405
Total elapsed time: 04h 43m 01s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1875/1875 [==============================] - 2046s 1s/step - loss: 0.3450 - accuracy: 0.9168
Epoch 2/10
1875/1875 [==============================] - 2036s 1s/step - loss: 0.1741 - accuracy: 0.9604
Epoch 3/10
1875/1875 [==============================] - 2039s 1s/step - loss: 0.2676 - accuracy: 0.9439
Epoch 4/10
1875/1875 [==============================] - 2040s 1s/step - loss: 0.1767 - accuracy: 0.9560
Epoch 5/10
1875/1875 [==============================] - 2045s 1s/step - loss: 0.2132 - accuracy: 0.9497
Epoch 6/10
1875/1875 [==============================] - 2043s 1s/step - loss: 0.1179 - accuracy: 0.9700
Epoch 7/10
1875/1875 [==============================] - 2046s 1s/step - loss: 0.1320 - accuracy: 0.9692
Epoch 8/10
1875/1875 [==============================] - 2049s 1s/step - loss: 0.0977 - accuracy: 0

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [5]:

input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [07h 48m 30s]
val_loss: 4.578865051269531

Best val_loss So Far: 4.578865051269531
Total elapsed time: 07h 48m 30s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
1875/1875 [==============================] - 3580s 2s/step - loss: 0.2054 - accuracy: 0.9427
Epoch 2/10
1875/1875 [==============================] - 3548s 2s/step - loss: 0.1375 - accuracy: 0.9671
Epoch 3/10
1875/1875 [==============================] - 4527s 2s/step - loss: 0.1028 - accuracy: 0.9740
Epoch 4/10
1875/1875 [==============================] - 3477s 2s/step - loss: 0.0775 - accuracy: 0.9800
Epoch 5/10
1875/1875 [==============================] - 3392s 2s/step - loss: 0.0682 - accuracy: 0.9813
Epoch 6/10
1875/1875 [==============================] - 3394s 2s/step - loss: 0.0642 - accuracy: 0.9837
Epoch 7/10
1875/1875 [==============================] - 3395s 2s/step - loss: 0.0425 - accuracy: 0.9880
Epoch 8/10
1875/1875 [==============================] - 5160s 3s/step - loss: 0.0364 - accuracy: 0.9894
Epoch 9/10
1875/1875 [==============================] - 5909s 3s/step - loss: 0.0310 - accuracy: 0.9911
Epoch 10/10
1875/1875 [==============================] - 5899s 3

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


# AutoKeras image classifier for STL10 
For train the image classifier on STL10 dataset 

In [1]:
import numpy as np
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from autokeras import ImageClassifier

# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

# Define the transform for the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the STL10 dataset using the DataLoader class
train_dataset = datasets.STL10(root='./data', split='train', transform=transform, download=True)
test_dataset = datasets.STL10(root='./data', split='test', transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=6, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=6, shuffle=False)

# Extract the data and labels from the loaders
x_train = []
y_train = []
for data, label in train_loader:
    x_train.append(data.numpy())
    y_train.append(label.numpy())
x_train = np.concatenate(x_train, axis=0)
y_train = np.concatenate(y_train, axis=0)

x_test = []
y_test = []
for data, label in test_loader:
    x_test.append(data.numpy())
    y_test.append(label.numpy())
x_test = np.concatenate(x_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

# Move the data to GPU if available
#x_train = torch.from_numpy(x_train).to(device)
#y_train = torch.from_numpy(y_train).to(device)
#x_test = torch.from_numpy(x_test).to(device)
#y_test = torch.from_numpy(y_test).to(device)

# Define the model
clf = ImageClassifier(max_trials=1, overwrite=True, seed=42)

# Train the model
clf.fit(x_train, y_train, epochs=10)

# Evaluate the model
score = clf.evaluate(x_test, y_test)
print('Test accuracy:', score[1])


Trial 1 Complete [00h 01m 29s]
val_loss: 1.6258485317230225

Best val_loss So Far: 1.6258485317230225
Total elapsed time: 00h 01m 29s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
157/157 [==============================] - 9s 54ms/step - loss: 1.9666 - accuracy: 0.2932
Epoch 2/10
157/157 [==============================] - 9s 54ms/step - loss: 1.7160 - accuracy: 0.3764
Epoch 3/10
157/157 [==============================] - 9s 57ms/step - loss: 1.6092 - accuracy: 0.4230
Epoch 4/10
157/157 [==============================] - 8s 52ms/step - loss: 1.5091 - accuracy: 0.4564
Epoch 5/10
157/157 [==============================] - 8s 51ms/step - loss: 1.4426 - accuracy: 0.4760
Epoch 6/10
157/157 [==============================] - 8s 53ms/step - loss: 1.3841 - accuracy: 0.4936
Epoch 7/10
157/157 [==============================] - 8s 54ms/step - loss: 1.3012 - accuracy: 0.5352
Epoch 8/10
157/157 [==============================] - 8s 50ms/step - loss: 1.2621 - accuracy: 0.5454
Epoch 9/10
157/157 [

INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


250/250 [==============================] - 5s 13ms/step - loss: 1.5838 - accuracy: 0.4634
Test accuracy: 0.4633750021457672


# AutoModel

In [ ]:
# Define the AutoModel
input_node = ak.ImageInput()
output_node = ak.ImageBlock()(input_node)
output_node = ak.Flatten()(output_node)
output_node = ak.DenseBlock()(output_node)
output_node = ak.ClassificationHead()(output_node)

# Initialize the AutoModel
clf = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=2
)

clf.fit(x_train, y_train, epochs=10)

# Evaluate the AutoModel on the test data
accuracy = clf.evaluate(x_test, y_test)[1]
print('Accuracy: {:.2f}%'.format(accuracy * 100))



Trial 2 Complete [00h 53m 39s]
val_loss: 2.166759729385376

Best val_loss So Far: 1.8759011030197144
Total elapsed time: 01h 46m 55s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
157/157 [==============================] - 403s 2s/step - loss: 2.3194 - accuracy: 0.1180
Epoch 2/10
157/157 [==============================] - 388s 2s/step - loss: 2.1743 - accuracy: 0.1734
Epoch 3/10
157/157 [==============================] - 396s 3s/step - loss: 2.0899 - accuracy: 0.1850
Epoch 4/10
157/157 [==============================] - 401s 3s/step - loss: 2.0313 - accuracy: 0.1864
Epoch 5/10
157/157 [==============================] - 389s 2s/step - loss: 1.9911 - accuracy: 0.1922
Epoch 6/10
157/157 [==============================] - 386s 2s/step - loss: 1.9605 - accuracy: 0.1922
Epoch 7/10
157/157 [==============================] - 347s 2s/step - loss: 1.9255 - accuracy: 0.1962
Epoch 8/10
157/157 [==============================] - 397s 3s/step - loss: 1.8821 - accuracy: 0.1950
Epoch 9/10
157/157 [==============================] - 403s 3s/step - loss: 1.8521 - accuracy: 0.1980
Epoch 10/10
157/157 [==============================] - 389s 2s/step - loss: 1.8342 - accura

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


# DTD

 # AutoModel
 Nueral architecture search with AutoModel on DTD dataset
 

In [3]:
import numpy as np
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from autokeras import ImageClassifier
# Check if a GPU is available, if not fall back to CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Define the transform for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # add resize operation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [4]:
# Load the DTD dataset using the DataLoader class
train_dataset = datasets.ImageFolder(root='/media/abdul/8266fdc5-61f6-47d6-9e82-aa0e56f4b319/Ilyas/classification/DTD/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/media/abdul/8266fdc5-61f6-47d6-9e82-aa0e56f4b319/Ilyas/classification/DTD/val', transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: torch.utils.data.dataloader.default_collate([y for y in x]))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: torch.utils.data.dataloader.default_collate([y for y in x]))

# Extract the data and labels from the loaders
x_train = np.concatenate([data.numpy() for data, label in train_loader], axis=0)
y_train = np.concatenate([label.numpy() for data, label in train_loader], axis=0)

x_test = np.concatenate([data.numpy() for data, label in test_loader], axis=0)
y_test = np.concatenate([label.numpy() for data, label in test_loader], axis=0)

In [5]:
import autokeras as ak
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=3
)
clf.fit(x_train, y_train, epochs=10)


Trial 3 Complete [00h 27m 28s]
val_loss: 3.960472345352173

Best val_loss So Far: 3.932887077331543
Total elapsed time: 01h 23m 25s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
93/93 [==============================] - 192s 2s/step - loss: 5.0663 - accuracy: 0.0216
Epoch 2/10
93/93 [==============================] - 180s 2s/step - loss: 3.8948 - accuracy: 0.0807
Epoch 3/10
93/93 [==============================] - 180s 2s/step - loss: 2.6727 - accuracy: 0.3171
Epoch 4/10
93/93 [==============================] - 180s 2s/step - loss: 1.5711 - accuracy: 0.7109
Epoch 5/10
93/93 [==============================] - 180s 2s/step - loss: 0.7831 - accuracy: 0.9477
Epoch 6/10
93/93 [==============================] - 179s 2s/step - loss: 0.5639 - accuracy: 0.9791
Epoch 7/10
93/93 [==============================] - 180s 2s/step - loss: 0.7277 - accuracy: 0.9500
Epoch 8/10
93/93 [==============================] - 180s 2s/step - loss: 0.7593 - accuracy: 0.9331
Epoch 9/10
93/93 [====================

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets
